<a href="https://colab.research.google.com/github/aml7hawaiiedu/CCAPLandCoverProject/blob/main/Train_Valid_split_CCAP_UNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# connect to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install rasterio scikit-image tensorflow keras gdown
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.5 MB/s eta 0:00:00


In [3]:
import rasterio
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
import os
import glob
import os
import gdown
import zipfile
from skimage.transform import resize
from tensorflow.keras import layers
import random
from keras.utils import to_categorical
from skimage.util import random_noise
from scipy import ndimage
from scipy.ndimage import label as nd_label
import cv2
from scipy.ndimage import generic_filter
from keras.models import load_model
from scipy.stats import mode
import glob
import os
import cv2
import numpy as np
import rasterio
from keras.utils import Sequence, to_categorical
import rasterio.plot
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
# Additional code can be added here if needed

In [4]:
# make a new folder
os.mkdir('/content/image_chips')

In [5]:
# Get all .zip files
zip_files = glob.glob('/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata/*.zip')

# direct to the new folder
destination_path = '/content/image_chips/'

for zip_file in zip_files:
    # Get the base name of the zip file (without .zip extension)
    base_name = os.path.basename(zip_file)[:-4]

    # Create a new directory for this zip file
    unzip_dir = os.path.join(destination_path, base_name)
    os.makedirs(unzip_dir, exist_ok=True)

    # Unzip into the specific directory
    !unzip -q "$zip_file" -d "$unzip_dir"

warning [/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata/Niihau_subset.zip]:  zipfile is empty
warning [/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata/Lanai_subset.zip]:  zipfile is empty


In [69]:
# Get all .csv files
csv_files = glob.glob('/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata/*.csv')


csv_list = []
for csv_file in csv_files:
    # Get the base name of the zip file (without .zip extension)
    base_name = os.path.basename(csv_file)[:-4]

    csv_loc = os.path.join(destination_path, base_name)
    base_csv = pd.read_csv(csv_file)
    base_csv['subset'] = base_name
    csv_list.append(base_csv)


index_csv = pd.concat(csv_list, ignore_index=True)

In [48]:
# make a new folder
os.mkdir('/content/index')


FileExistsError: ignored

In [49]:
# Compute total sum of percent values for each label
print(index_csv.groupby('label')['percent'].sum())

label
13    339.165585
14     80.866539
15     26.158833
16      5.413750
17      3.640327
18      4.082653
Name: percent, dtype: float64


In [50]:
# Group by label and count the number of occurances of each label
# how many rows are associated with each label in the index?
print(index_csv.groupby('label')['percent'].count())

label
13    1431
14    1117
15    1285
16     199
17     205
18     170
Name: percent, dtype: int64


In [51]:
# Compute the average percent value of each label category. Useful for calculating some kind of weighted average, or average per category
index_csv.groupby('label')['percent'].sum()/index_csv.groupby('label')['percent'].count()

label
13    0.237013
14    0.072396
15    0.020357
16    0.027205
17    0.017758
18    0.024016
Name: percent, dtype: float64

In [70]:
index_csv

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent,tif_name,subset
0,9217,1025,8705,513,61,438833.983073,440062.758637,2.457574e+06,2.458802e+06,13,0.000919,8705_513.tif,Kauai10_subset
1,9729,1025,9217,513,62,440062.834396,441291.605845,2.457574e+06,2.458802e+06,13,0.033161,9217_513.tif,Kauai10_subset
2,13825,1025,13313,513,70,449893.209832,451122.010258,2.457574e+06,2.458802e+06,13,0.000130,13313_513.tif,Kauai10_subset
3,13825,1025,13313,513,70,449893.209832,451122.010258,2.457574e+06,2.458802e+06,14,0.000137,13313_513.tif,Kauai10_subset
4,13825,1025,13313,513,70,449893.209832,451122.010258,2.457574e+06,2.458802e+06,15,0.003502,13313_513.tif,Kauai10_subset
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4402,1537,1025,1025,513,26,703456.408765,704685.207789,2.313662e+06,2.314891e+06,15,0.000828,1025_513.tif,Lanai_subset
4403,1537,1025,1025,513,26,703456.420411,704685.233874,2.313662e+06,2.314891e+06,15,0.000828,1025_513.tif,Lanai11_subset
4404,7169,5121,6657,4609,221,716973.164648,718201.965448,2.303832e+06,2.305061e+06,15,0.001396,6657_4609.tif,Lanai11_subset
4405,7681,8705,7169,8193,383,718201.993245,719430.775154,2.295230e+06,2.296459e+06,15,0.001637,7169_8193.tif,Lanai11_subset


In [59]:
index_csv.groupby(['tif_name', 'subset']).size().reset_index(name='Count')

,tif_name,subset,Count
0,10241_10241.tif,Oahu11_subset,1
1,10241_1025.tif,Kauai10_subset,2
2,10241_1025.tif,Kauai_subset,2
3,10241_10753.tif,Maui10_subset,2
4,10241_10753.tif,Maui_subset,2
...,...,...,...
2350,9729_7681.tif,Oahu_subset,1
2351,9729_8193.tif,Kauai10_subset,1
2352,9729_8193.tif,Kauai_subset,1
2353,9729_9729.tif,Oahu11_subset,2


In [78]:
# Calculate the number of rows for training (80%) and validation (20%)
total_rows = len(index_csv)
train_rows = int(0.8 * total_rows)
val_rows = total_rows - train_rows

# Generate an array of 0s (80% of total) and 1s (20% of total) randomly
random_assignment = np.array([0] * train_rows + [1] * val_rows)

# Shuffle the array to randomize the assignment
np.random.shuffle(random_assignment)

# Add the random assignment as 'split' column to your DataFrame
index_csv['random_split'] = random_assignment

# # Save the updated DataFrame to a CSV file
# index_csv.to_csv('index_random.csv', index=False)

print("Random split column added to the DataFrame.")


Random split column added to the DataFrame.


In [79]:
index_csv

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent,tif_name,subset,random_split
0,9217,1025,8705,513,61,438833.983073,440062.758637,2.457574e+06,2.458802e+06,13,0.000919,8705_513.tif,Kauai10_subset,0
1,9729,1025,9217,513,62,440062.834396,441291.605845,2.457574e+06,2.458802e+06,13,0.033161,9217_513.tif,Kauai10_subset,1
2,13825,1025,13313,513,70,449893.209832,451122.010258,2.457574e+06,2.458802e+06,13,0.000130,13313_513.tif,Kauai10_subset,0
3,13825,1025,13313,513,70,449893.209832,451122.010258,2.457574e+06,2.458802e+06,14,0.000137,13313_513.tif,Kauai10_subset,0
4,13825,1025,13313,513,70,449893.209832,451122.010258,2.457574e+06,2.458802e+06,15,0.003502,13313_513.tif,Kauai10_subset,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4402,1537,1025,1025,513,26,703456.408765,704685.207789,2.313662e+06,2.314891e+06,15,0.000828,1025_513.tif,Lanai_subset,1
4403,1537,1025,1025,513,26,703456.420411,704685.233874,2.313662e+06,2.314891e+06,15,0.000828,1025_513.tif,Lanai11_subset,1
4404,7169,5121,6657,4609,221,716973.164648,718201.965448,2.303832e+06,2.305061e+06,15,0.001396,6657_4609.tif,Lanai11_subset,0
4405,7681,8705,7169,8193,383,718201.993245,719430.775154,2.295230e+06,2.296459e+06,15,0.001637,7169_8193.tif,Lanai11_subset,1


In [85]:
# Define the path for the CSV file
csv_output_path = '/content/index/index_random.csv'

# Export the DataFrame to a CSV file
index_csv.to_csv(csv_output_path, index=False)

print(f'DataFrame exported to {csv_output_path}')

DataFrame exported to /content/index/index_random.csv


In [86]:
# Load your updated DataFrame with the 'split' column
index_csv = pd.read_csv('updated_index.csv')  # Replace with your actual file path

# Count the occurrences of '0' and '1' in the 'split' column
split_counts = index_csv['random_split'].value_counts()

# Print the counts
print("Count of files assigned '0':", split_counts.get(0, 0))
print("Count of files assigned '1':", split_counts.get(1, 0))

Count of files assigned '0': 3525
Count of files assigned '1': 882


In [87]:
index_csv

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent,tif_name,subset,random_split
0,9217,1025,8705,513,61,438833.983073,440062.758637,2.457574e+06,2.458802e+06,13,0.000919,8705_513.tif,Kauai10_subset,0
1,9729,1025,9217,513,62,440062.834396,441291.605845,2.457574e+06,2.458802e+06,13,0.033161,9217_513.tif,Kauai10_subset,1
2,13825,1025,13313,513,70,449893.209832,451122.010258,2.457574e+06,2.458802e+06,13,0.000130,13313_513.tif,Kauai10_subset,1
3,13825,1025,13313,513,70,449893.209832,451122.010258,2.457574e+06,2.458802e+06,14,0.000137,13313_513.tif,Kauai10_subset,0
4,13825,1025,13313,513,70,449893.209832,451122.010258,2.457574e+06,2.458802e+06,15,0.003502,13313_513.tif,Kauai10_subset,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4402,1537,1025,1025,513,26,703456.408765,704685.207789,2.313662e+06,2.314891e+06,15,0.000828,1025_513.tif,Lanai_subset,0
4403,1537,1025,1025,513,26,703456.420411,704685.233874,2.313662e+06,2.314891e+06,15,0.000828,1025_513.tif,Lanai11_subset,0
4404,7169,5121,6657,4609,221,716973.164648,718201.965448,2.303832e+06,2.305061e+06,15,0.001396,6657_4609.tif,Lanai11_subset,1
4405,7681,8705,7169,8193,383,718201.993245,719430.775154,2.295230e+06,2.296459e+06,15,0.001637,7169_8193.tif,Lanai11_subset,0


UPDATE BELOW. Stopping here for now.

In [ ]:
# Load your DataFrame (replace 'updated_index.csv' with your actual file path)
index_csv = pd.read_csv('updated_index.csv')

# Define the source directory where image and label files are located
source_directory = '/path/to/source_directory'  # Replace with the actual path

# Define the destination directories for training and validation
training_destination = '/path/to/training_destination'  # Replace with your desired path
validation_destination = '/path/to/validation_destination'  # Replace with your desired path

# Create the training and validation directories if they don't exist
os.makedirs(training_destination, exist_ok=True)
os.makedirs(validation_destination, exist_ok=True)

# Iterate through each row in the DataFrame
for _, row in index_csv.iterrows():
    # Construct the source and destination paths for image files
    source_image_path = os.path.join(source_directory, 'Images', row['tif_name'])
    dest_image_folder = training_destination if row['split'] == 0 else validation_destination
    dest_image_path = os.path.join(dest_image_folder, 'Images', row['tif_name'])

    # Construct the source and destination paths for label files
    source_label_path = os.path.join(source_directory, 'Labels', row['tif_name'])
    dest_label_folder = training_destination if row['split'] == 0 else validation_destination
    dest_label_path = os.path.join(dest_label_folder, 'Labels', row['tif_name'])

    # Copy the image and label files to their respective destinations
    shutil.copy(source_image_path, dest_image_path)
    shutil.copy(source_label_path, dest_label_path)

print("Files separated into training and validation folders.")
